<a href="https://colab.research.google.com/github/SALMONPreet/stuff1/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np

# Reload data to ensure 'expenses' column exists
raw_dataset = pd.read_csv('insurance.csv')

# Separate label
labels = raw_dataset.pop('expenses')

# Train-test split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(
    raw_dataset, labels, test_size=0.2, random_state=42
)

# Define feature types
categorical = ['sex', 'smoker', 'region']
numeric = ['age', 'bmi', 'children']

# Preprocessor: scale numerics, one-hot encode categoricals
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric),
    ('cat', OneHotEncoder(sparse_output=False), categorical)
])

# Fit-transform data
X_train = preprocessor.fit_transform(train_dataset)
X_test = preprocessor.transform(test_dataset)

# Get final column names
encoded_cat_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical)
all_columns = np.concatenate([numeric, encoded_cat_columns])

# Convert arrays back to DataFrames so the test cell works
train_dataset = pd.DataFrame(X_train, columns=all_columns)
test_dataset = pd.DataFrame(X_test, columns=all_columns)

# Build model
model = keras.Sequential([
    layers.Input(shape=(train_dataset.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mae', metrics=['mae', 'mse'])

# Train model
history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
